# ---------------------------------------------------------
# 🚨 MASTER SETUP BLOCK - DO NOT EDIT THIS CELL
# ---------------------------------------------------------

In [ ]:
# ==============================================================================
# 🚀 MASTER DATA PIPELINE (LOAD -> CLEAN -> FILL)
# ==============================================================================
# Instructions:
# 1. Place 'US_Accidents_March23.csv' in a folder named 'data'.
# 2. Run the cells below in order to get a clean dataframe ready for analysis.
# ==============================================================================

import pandas as pd
import numpy as np
import os

In [ ]:
# ---------------------------------------------------------
# 1. LOAD DATA
# ---------------------------------------------------------
file_path = os.path.join('data', 'US_Accidents_March23.csv')
print("⏳ Loading dataset (Sample Mode)...")
# ⚠️ SAFETY: Loading 500k rows for development.
# To run on the FULL dataset for the final report, remove 'nrows=500000'
df = pd.read_csv(file_path, nrows=500000)
print(f"✅ Loaded {df.shape[0]} rows.")

In [ ]:
# ---------------------------------------------------------
# 2. DROP USELESS COLUMNS
# ---------------------------------------------------------
# We keep 'Amenity' as requested, but drop other noise.
cols_to_delete = [
    # Metadata & IDs
    'ID', 'Source', 'Country', 'Zipcode', 'Timezone', 'Airport_Code', 'Description',

    # Redundant Time/Weather
    'Weather_Timestamp', 'Wind_Chill(F)', 'Wind_Direction', 'Pressure(in)', 'Precipitation(in)',
    'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight',

    # Too Granular or Mostly False
    'Street', 'Turning_Loop',

    # High Missing Values
    'End_Lat', 'End_Lng'
]
df.drop(columns=cols_to_delete, errors='ignore', inplace=True)
print(f"🗑️ Dropped {len(cols_to_delete)} columns.")

In [ ]:
# ---------------------------------------------------------
# 3. RENAME COLUMNS (For easier coding)
# ---------------------------------------------------------
df.rename(columns={
    'Distance(mi)': 'Distance',
    'Temperature(F)': 'Temp',
    'Humidity(%)': 'Humidity',
    'Visibility(mi)': 'Visibility',
    'Wind_Speed(mph)': 'Wind_Speed',
    'Weather_Condition': 'Weather'
}, inplace=True)

In [ ]:
# ---------------------------------------------------------
# 4. FIX DATE & TIME
# ---------------------------------------------------------
print("🕒 Converting timestamps and calculating duration...")
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

# Drop rows where Start_Time or End_Time is unknown (cannot analyze without time)
df.dropna(subset=['Start_Time', 'End_Time'], inplace=True)

# Create "Duration" (in Minutes)
df['Duration'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds() / 60

# Filter logic: Remove negative durations or accidents lasting > 1 week (bad data)
df = df[(df['Duration'] > 0) & (df['Duration'] < 10080)]

# Extract Temporal Features
df['Year'] = df['Start_Time'].dt.year
df['Month'] = df['Start_Time'].dt.month
df['Hour'] = df['Start_Time'].dt.hour
df['Weekday'] = df['Start_Time'].dt.day_name()

In [ ]:
# ---------------------------------------------------------
# 5. FIX MISSING VALUES (IMPUTATION)
# ---------------------------------------------------------
print("🔧 Filling empty values...")
# A. Numerical Columns -> Fill with MEDIAN
weather_nums = ['Temp', 'Humidity', 'Visibility', 'Wind_Speed']
for col in weather_nums:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# B. Categorical Columns -> Fill with MODE (Most Frequent)
categorical_cols = ['Weather', 'Sunrise_Sunset', 'City']
for col in categorical_cols:
    if col in df.columns:
        if not df[col].mode().empty:
            mode_val = df[col].mode()[0]
            df[col] = df[col].fillna(mode_val)

In [ ]:
# ---------------------------------------------------------
# 6. FINAL STATUS CHECK
# ---------------------------------------------------------
print("\n✅ DATA PIPELINE COMPLETE!")
print(f"Final Shape: {df.shape}")
print(f"Any missing values left? {df.isna().sum().sum()}")
print("------------------------------------------------")
print("Columns ready for analysis:", list(df.columns))
df.head(3)